# Instance Normalization - Time Series Data

## Giới thiệu

Instance Normalization cho dữ liệu time series với shape (batch, time_steps, features).

Đối với time series phức tạp, shape có thể là: (batch_size, time, V*len(lag)+number_of_time)
- V: số biến (variables)
- len(lag): độ dài của lag window
- number_of_time: số features thời gian bổ sung

Instance Normalization sẽ normalize theo:
- Mean và variance được tính trên time dimension
- Normalize cho từng sample và từng feature độc lập

## Công thức

```
mean = mean(x, dim=1)  # Tính mean theo time dim
var = var(x, dim=1)    # Tính variance theo time dim
x_norm = (x - mean) / sqrt(var + eps)
output = gamma * x_norm + beta
```

Trong đó:
- gamma: learnable scale parameter (một cho mỗi feature)
- beta: learnable shift parameter (một cho mỗi feature)
- eps: small constant để tránh chia cho 0

Khác với BatchNorm và LayerNorm: InstanceNorm normalize từng feature riêng biệt cho từng sample.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class InstanceNorm1D_TS:
    """
    Instance Normalization cho time series data (3D tensor: batch, time_steps, features)
    Hỗ trợ shape: (batch_size, time, V*len(lag)+number_of_time)
    """
    def __init__(self, num_features, eps=1e-5):
        self.num_features = num_features
        self.eps = eps
        
        # Learnable parameters
        self.gamma = np.ones(num_features)  # Scale
        self.beta = np.zeros(num_features)  # Shift
        
    def forward(self, x):
        """
        x shape: (batch, time_steps, features)
        Có thể là shape phức tạp: (batch_size, time, V*len(lag)+number_of_time)
        """
        # Tính mean và var theo time dimension
        # Normalize cho từng sample và từng feature độc lập
        mean = np.mean(x, axis=1, keepdims=True)
        var = np.var(x, axis=1, keepdims=True)
        
        # Normalize
        x_norm = (x - mean) / np.sqrt(var + self.eps)
        
        # Scale and shift
        gamma = self.gamma.reshape(1, 1, -1)
        beta = self.beta.reshape(1, 1, -1)
        output = gamma * x_norm + beta
        
        return output

print("InstanceNorm1D_TS class đã được định nghĩa!")


## Ví dụ 1: 1 Sample, 1 Feature

Shape: (1, 10, 1) - 1 batch, 10 time steps, 1 feature


In [ ]:
# Ví dụ 1: 1 Sample, 1 Feature
np.random.seed(42)
t = np.linspace(0, 4*np.pi, 10)
x1 = np.sin(t).reshape(1, 10, 1) * 5 + np.random.randn(1, 10, 1) * 2
print(f"Input shape: {x1.shape}")
print(f"Input mean: {np.mean(x1[0, :, 0]):.4f}, std: {np.std(x1[0, :, 0]):.4f}")

in1 = InstanceNorm1D_TS(num_features=1)
output1 = in1.forward(x1)
print(f"\nOutput mean: {np.mean(output1[0, :, 0]):.4f}, std: {np.std(output1[0, :, 0]):.4f}")

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].plot(x1[0, :, 0], marker='o')
axes[0].set_title('Input (Before IN)')
axes[0].set_xlabel('Time Step')
axes[0].set_ylabel('Value')
axes[0].grid(True)
axes[1].plot(output1[0, :, 0], marker='o', color='orange')
axes[1].set_title('Output (After IN)')
axes[1].set_xlabel('Time Step')
axes[1].set_ylabel('Value')
axes[1].grid(True)
plt.tight_layout()
plt.show()


## Ví dụ 2: 1 Sample, 2 Features

Shape: (1, 10, 2) - 1 batch, 10 time steps, 2 features


In [ ]:
# Ví dụ 2: 1 Sample, 2 Features
np.random.seed(42)
t = np.linspace(0, 4*np.pi, 10)
x2 = np.zeros((1, 10, 2))
x2[0, :, 0] = np.sin(t) * 5 + np.random.randn(10) * 2
x2[0, :, 1] = np.cos(t) * 3 + np.random.randn(10) * 1.5
print(f"Input shape: {x2.shape}")
print(f"Feature 0 mean: {np.mean(x2[0, :, 0]):.4f}, std: {np.std(x2[0, :, 0]):.4f}")
print(f"Feature 1 mean: {np.mean(x2[0, :, 1]):.4f}, std: {np.std(x2[0, :, 1]):.4f}")

in2 = InstanceNorm1D_TS(num_features=2)
output2 = in2.forward(x2)
print(f"\nFeature 0 - Output mean: {np.mean(output2[0, :, 0]):.4f}, std: {np.std(output2[0, :, 0]):.4f}")
print(f"Feature 1 - Output mean: {np.mean(output2[0, :, 1]):.4f}, std: {np.std(output2[0, :, 1]):.4f}")

fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes[0, 0].plot(x2[0, :, 0], marker='o')
axes[0, 0].set_title('Feature 0 Input')
axes[0, 0].grid(True)
axes[0, 1].plot(x2[0, :, 1], marker='o', color='green')
axes[0, 1].set_title('Feature 1 Input')
axes[0, 1].grid(True)
axes[1, 0].plot(output2[0, :, 0], marker='o', color='orange')
axes[1, 0].set_title('Feature 0 Output')
axes[1, 0].grid(True)
axes[1, 1].plot(output2[0, :, 1], marker='o', color='red')
axes[1, 1].set_title('Feature 1 Output')
axes[1, 1].grid(True)
plt.tight_layout()
plt.show()


## Ví dụ 3: 2 Samples, 1 Feature

Shape: (2, 10, 1) - 2 batches, 10 time steps, 1 feature


In [ ]:
# Ví dụ 3: 2 Samples, 1 Feature
np.random.seed(42)
t = np.linspace(0, 4*np.pi, 10)
x3 = np.zeros((2, 10, 1))
x3[0, :, 0] = np.sin(t) * 5 + np.random.randn(10) * 2
x3[1, :, 0] = np.cos(t) * 3 + np.random.randn(10) * 1.5
print(f"Input shape: {x3.shape}")
print(f"Sample 0 mean: {np.mean(x3[0, :, 0]):.4f}, std: {np.std(x3[0, :, 0]):.4f}")
print(f"Sample 1 mean: {np.mean(x3[1, :, 0]):.4f}, std: {np.std(x3[1, :, 0]):.4f}")

in3 = InstanceNorm1D_TS(num_features=1)
output3 = in3.forward(x3)
print(f"\nSample 0 - Output mean: {np.mean(output3[0, :, 0]):.4f}, std: {np.std(output3[0, :, 0]):.4f}")
print(f"Sample 1 - Output mean: {np.mean(output3[1, :, 0]):.4f}, std: {np.std(output3[1, :, 0]):.4f}")

fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes[0, 0].plot(x3[0, :, 0], marker='o')
axes[0, 0].set_title('Sample 0 Input')
axes[0, 0].grid(True)
axes[0, 1].plot(x3[1, :, 0], marker='o', color='green')
axes[0, 1].set_title('Sample 1 Input')
axes[0, 1].grid(True)
axes[1, 0].plot(output3[0, :, 0], marker='o', color='orange')
axes[1, 0].set_title('Sample 0 Output')
axes[1, 0].grid(True)
axes[1, 1].plot(output3[1, :, 0], marker='o', color='red')
axes[1, 1].set_title('Sample 1 Output')
axes[1, 1].grid(True)
plt.tight_layout()
plt.show()


## Ví dụ 4: Shape phức tạp - (batch_size, time, V*len(lag)+number_of_time)

Ví dụ với:
- batch_size = 2
- time = 10
- V = 2 (số biến)
- len(lag) = 3 (lag window)
- number_of_time = 2 (features thời gian bổ sung)
- Total features = V*len(lag) + number_of_time = 2*3 + 2 = 8

Shape: (2, 10, 8)


In [ ]:
# Ví dụ 4: Shape phức tạp (batch_size, time, V*len(lag)+number_of_time)
np.random.seed(42)

# Tham số
batch_size = 2
time = 10
V = 2  # số biến
len_lag = 3  # độ dài lag window
number_of_time = 2  # features thời gian bổ sung
num_features = V * len_lag + number_of_time  # 2*3 + 2 = 8

print(f"Tham số:")
print(f"  batch_size = {batch_size}")
print(f"  time = {time}")
print(f"  V = {V} (số biến)")
print(f"  len(lag) = {len_lag} (lag window)")
print(f"  number_of_time = {number_of_time}")
print(f"  Total features = V*len(lag) + number_of_time = {V}*{len_lag} + {number_of_time} = {num_features}")

# Tạo dữ liệu mô phỏng
t = np.linspace(0, 4*np.pi, time)
x4 = np.zeros((batch_size, time, num_features))

for b in range(batch_size):
    # Features từ lag windows (V*len_lag features)
    for v in range(V):
        base_signal = np.sin((v+1)*t) * (5 + b*2) + np.random.randn(time) * 2
        for lag in range(len_lag):
            feature_idx = v * len_lag + lag
            x4[b, :, feature_idx] = np.roll(base_signal, lag) + np.random.randn(time) * 0.5
    
    # Features thời gian bổ sung (number_of_time features)
    x4[b, :, V*len_lag] = np.cos(t) * 3 + np.random.randn(time) * 1.5
    x4[b, :, V*len_lag+1] = t / np.max(t) * 10 + np.random.randn(time) * 1

print(f"\nInput shape: {x4.shape}")

# Khởi tạo InstanceNorm với số features phức tạp
in4 = InstanceNorm1D_TS(num_features=num_features)
output4 = in4.forward(x4)

print(f"\nOutput shape: {output4.shape}")

# Hiển thị statistics cho một vài features
print(f"\nSample 0 - Feature 0 (V0, lag0): Input mean={np.mean(x4[0, :, 0]):.4f}, Output mean={np.mean(output4[0, :, 0]):.4f}")
print(f"Sample 0 - Feature 3 (V1, lag0): Input mean={np.mean(x4[0, :, 3]):.4f}, Output mean={np.mean(output4[0, :, 3]):.4f}")
print(f"Sample 0 - Feature 6 (time feature 0): Input mean={np.mean(x4[0, :, 6]):.4f}, Output mean={np.mean(output4[0, :, 6]):.4f}")

# Visualize một vài features
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes[0, 0].plot(x4[0, :, 0], marker='o')
axes[0, 0].set_title('Sample 0, Feature 0 (V0, lag0) Input')
axes[0, 0].grid(True)
axes[0, 1].plot(x4[0, :, 3], marker='o', color='green')
axes[0, 1].set_title('Sample 0, Feature 3 (V1, lag0) Input')
axes[0, 1].grid(True)
axes[0, 2].plot(x4[0, :, 6], marker='o', color='blue')
axes[0, 2].set_title('Sample 0, Feature 6 (time) Input')
axes[0, 2].grid(True)

axes[1, 0].plot(output4[0, :, 0], marker='o', color='orange')
axes[1, 0].set_title('Sample 0, Feature 0 Output')
axes[1, 0].grid(True)
axes[1, 1].plot(output4[0, :, 3], marker='o', color='red')
axes[1, 1].set_title('Sample 0, Feature 3 Output')
axes[1, 1].grid(True)
axes[1, 2].plot(output4[0, :, 6], marker='o', color='purple')
axes[1, 2].set_title('Sample 0, Feature 6 Output')
axes[1, 2].grid(True)

plt.tight_layout()
plt.show()
